# Challenge two: Programming a RAG system in BigQuery
Goal: Demonstrate your ability to program a RAG system that uses BigQuery to generate embeddings and perform a vector search.

### Install required packages

In [ ]:
!pip install langchain-google-genai langchain-google-vertexai langchain-google-community langchain-core --quiet

### Import required packages

In [ ]:
from google.cloud import bigquery

### Setup variables

In [ ]:
PROJECT_ID='qwiklabs-gcp-03-7a8bdf6e2e2c'
LOCATION='us'
DATASET = "AuroraBay"
TABLE = "faqs"
TABLE_EMBEDDED = "faqs_embedded"

In [ ]:
# Construct a BigQuery client object.
client = bigquery.Client(project=PROJECT_ID)

### Create a new dataset in BigQuery

In [ ]:
# Create dataset
dataset_id = "{}.{}".format(client.project, DATASET)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# Specify the geographic location where the dataset should reside.
dataset.location = "US"

dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

### Create a new table

In [ ]:
# Create table
table_id = "{}.{}.{}".format(client.project, dataset.dataset_id, TABLE)

table = bigquery.Table(table_id)
table = client.create_table(table)  # API request

print(f"Created {table_id}.")

### Load CSV into table

In [ ]:
# Load data from CSV
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("question", "STRING"),
        bigquery.SchemaField("answer", "STRING"),
    ],
    skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
uri = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

### Create a Cloud resource connection

In [ ]:
!bq mk --connection --connection_type=CLOUD_RESOURCE --location=us --project_id={PROJECT_ID} "embedding_conn"
!bq show --location=us --connection --project_id={PROJECT_ID} "embedding_conn"

In [ ]:
# Update you service acccount here
connection_service_account = "bqcx-569779670169-huh6@gcp-sa-bigquery-condel.iam.gserviceaccount.com" # @param {"type": "string"}
connection_member = f"serviceAccount:{connection_service_account}"

!gcloud projects add-iam-policy-binding {PROJECT_ID} --member={connection_member} --role='roles/aiplatform.user' --condition=None --quiet
# !gcloud projects add-iam-policy-binding {PROJECT_ID} --member={connection_member} --role='roles/bigquery.dataowner' --condition=None --quiet

### Create embedding model

In [ ]:
query = f"""
CREATE OR REPLACE MODEL `AuroraBay.Embeddings`
REMOTE WITH CONNECTION `us.embedding_conn` OPTIONS (ENDPOINT = 'text-embedding-005');
"""

query_job = client.query(query)  # API request
query_job.result()  # Waits for the query to complete

print("Embeddings table created successfully.")


### Generate embeddings

In [ ]:
query = f"""
CREATE OR REPLACE TABLE `AuroraBay.faqs_embedded` AS SELECT *
FROM ML.GENERATE_EMBEDDING(
    MODEL `AuroraBay.Embeddings`,
(SELECT CONCAT(question, ' ', answer) content FROM `AuroraBay.faqs`)
);
"""

query_job = client.query(query)  # API request
query_job.result()  # Waits for the query to complete

print("Embeddings generated successfully.")

## Langchain setup

### Create an embedding class instance

In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings

embedding = VertexAIEmbeddings(
    model_name="text-embedding-005", project=PROJECT_ID
)

### Initialize BigQueryVectorStore

In [ ]:
from langchain_google_community import BigQueryVectorStore

store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE_EMBEDDED,
    location=LOCATION,
    embedding=embedding,
    embedding_field="ml_generate_embedding_result",
)

### Compose a LangChain Chain

In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_vertexai import VertexAI

# See full prompt at https://smith.langchain.com/hub/rlm/rag-prompt
prompt = hub.pull("langchain-ai/retrieval-qa-chat")
llm = VertexAI(model_name="gemini-2.0-flash")

qa_chain = (
    {
        "context": store.as_retriever(),
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
qa_chain.invoke("what is 1 + 1?")

In [ ]:
# Chat loop
while True:
  user_input = input("You: ")
  if user_input.lower().strip() in ['exit', 'quit']:
        print("👋 Bye!")
        break
  print("Bot:")
  print(qa_chain.invoke(user_input))